In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import re
import datetime
import os.path

* 层次化索引
* 数据的联合及合并
* 重塑及转轴

## 一、层次化索引
> 层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理高维度数据。

In [2]:
data = pd.Series(np.random.randn(9),
   ...:                  index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
   ...:                         [1, 2, 3, 1, 3, 1, 2, 2, 3]])


In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

数据选取

In [4]:
data['a':'b']

a  1   -1.397390
   2   -1.641580
   3   -1.552073
b  1    1.785600
   3   -0.294151
dtype: float64

In [6]:
data.loc[:,[2,3]]

a  2   -1.641580
   3   -1.552073
b  3   -0.294151
c  2    0.089667
d  2   -0.779014
   3   -0.286791
dtype: float64

数据堆叠及展开

In [8]:
data.unstack().stack()

a  1   -1.397390
   2   -1.641580
   3   -1.552073
b  1    1.785600
   3   -0.294151
c  1    0.786713
   2    0.089667
d  2   -0.779014
   3   -0.286791
dtype: float64

In [9]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
   ....:                      index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
   ....:                      columns=[['Ohio', 'Ohio', 'Colorado'],
   ....:                               ['Green', 'Red', 'Green']])

In [11]:
frame.loc['a',:]

Ohio     Colorado
  Green Red    Green
1     0   1        2
2     3   4        5

In [13]:
#创建层次索引以便复用
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### 重排与分级排序

In [14]:
frame.swaplevel(0,1)

Ohio     Colorado
    Green Red    Green
1 a     0   1        2
2 a     3   4        5
1 b     6   7        8
2 b     9  10       11

In [16]:
frame.sort_index(level = 1)

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
b 1     6   7        8
a 2     3   4        5
b 2     9  10       11

### 根据级别汇总统计

In [17]:
frame.sum(level = 1)

Ohio     Colorado
  Green Red    Green
1     6   8       10
2    12  14       16

In [18]:
frame.sum(level = 1,axis = 1)

Green  Red
a 1      2    1
  2      8    4
b 1     14    7
  2     20   10

### 列索引

In [19]:
frame1 = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
   ....:                       'c': ['one', 'one', 'one', 'two', 'two',
   ....:                             'two', 'two'],
   ....:                       'd': [0, 1, 2, 0, 1, 2, 3]})


In [20]:
frame2 = frame1.set_index('c')

In [21]:
frame2

,a,b,d
c,,,
one,0,7,0
one,1,6,1
one,2,5,2
two,3,4,0
two,4,3,1
two,5,2,2
two,6,1,3


In [23]:
frame2.reset_index().reindex(['a','b','c','d'],axis =1)

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


> 合并，索引将会被丢弃；
> 重复列名将会被重命名

In [3]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
   ....:                      'key2': ['one', 'two', 'one'],
   ....:                      'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
   ....:                       'key2': ['one', 'one', 'one', 'two'],
   ....:                       'rval': [4, 5, 6, 7]})


In [6]:
frame8 = pd.merge(left,right,on = 'key1',suffixes = ('_left','_right'))

In [7]:
frame8

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 索引上的合并
> left_index,right_index

#### DataFrame还有一个便捷的join实例方法，它能更为方便地实现按索引合并

In [8]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
   ....:                      index=['a', 'c', 'e'],
   ....:                      columns=['Ohio', 'Nevada'])

In [69]: right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
   ....:                       index=['b', 'c', 'd', 'e'],
   ....:                       columns=['Missouri', 'Alabama'])


In [9]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


### 轴向连接
>另一种数据合并运算也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）。NumPy的concatenation函数可以用NumPy数组来做：

In [10]:
arr = np.arange(12).reshape((3, 4))


In [13]:
np.concatenate((arr,arr))

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [14]:
In [82]: s1 = pd.Series([0, 1], index=['a', 'b'])

In [83]: s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

In [84]: s3 = pd.Series([5, 6], index=['f', 'g'])


In [16]:
pd.concat([s1,s2,s3],axis = 1,join = 'inner')

,0,1,2


### 合并重叠数据
combine_first使用NumPy的where函数，它表示一种等价于面向数组的if-else

In [10]:
In [108]: a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
   .....:               index=['f', 'e', 'd', 'c', 'b', 'a'])

In [109]: b = pd.Series(np.arange(len(a), dtype=np.float64),
   .....:               index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan



In [15]:
a[-1] = 10

In [12]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [16]:
b[:-2].combine_first(a[2:])

a    10.0
b     4.5
c     3.0
d     2.0
e     1.0
f     0.0
dtype: float64

In [4]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [17]:
In [115]: df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
   .....:                     'b': [np.nan, 2., np.nan, 6.],
   .....:                     'c': range(2, 18, 4)})

In [116]: df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
   .....:                     'b': [np.nan, 3., 4., 6., 8.]})


In [18]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 重塑和轴向旋转
> 有许多用于重新排列表格型数据的基础运算。这些函数也称作重塑（reshape）或轴向旋转（pivot）运算。

### 重塑层次化索引
* stack 将数据的列重塑为行
* unstack 将数据的列重塑为列

In [19]:
In [120]: data = pd.DataFrame(np.arange(6).reshape((2, 3)),
   .....:                     index=pd.Index(['Ohio','Colorado'], ame='state'),
   .....:                     columns=pd.Index(['one', 'two', 'three'],
   .....:                     name='number'))



In [20]:
data

number,one,two,three
Ohio,0,1,2
Colorado,3,4,5


In [23]:
data.stack().unstack(level = 0)
#成为内层索引

,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


### 将“长格式”旋转为“宽格式”

In [29]:
data9 = pd.read_csv('e:\jupyter\pydata-book\examples\macrodata.csv')

In [31]:
data9.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [32]:
periods = pd.PeriodIndex(year = data9.year,quarter = data9.quarter,name = 'date')

In [33]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [35]:
data9 = data9.reindex(columns = columns)

In [36]:
data9.index = periods.to_timestamp('D', 'end')

In [65]:
ldata = data9.stack()

In [66]:
ldata = ldata.reset_index().rename(columns = {0:'value'})

In [67]:
ldata.head()

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340


In [68]:
pivoted = ldata.pivot(index = 'date',columns = 'item',values = 'value')

In [47]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [48]:
ldata['value2'] = np.random.randn(len(ldata))

In [49]:
pivoted2 = ldata.pivot('date','item')

In [50]:
pivoted2[:10]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.437877 -1.472093  1.721358
1959-06-30  2.34  2778.801   5.1  0.958215  1.617037 -0.685642
1959-09-30  2.74  2775.488   5.3  1.466274 -1.503125 -1.247412
1959-12-31  0.27  2785.204   5.6 -0.350741  0.298730 -1.938654
1960-03-31  2.31  2847.699   5.2 -0.544541 -1.201026 -0.245193
1960-06-30  0.14  2834.390   5.2 -0.207895 -2.083533 -1.136374
1960-09-30  2.70  2839.022   5.6 -0.648102  0.385712  0.712359
1960-12-31  1.21  2802.616   6.3  0.750663 -1.049285  0.591678
1961-03-31 -0.40  2819.264   6.8 -2.729431  0.317933  0.486536
1961-06-30  1.47  2872.005   7.0 -0.542326 -0.271943 -0.193465

In [55]:
pivoted2 = ldata.set_index(['date','item']).unstack('item')

In [56]:
pivoted2

value                     value2                    
item        infl    realgdp unemp      infl   realgdp     unemp
date                                                           
1959-03-31  0.00   2710.349   5.8  0.437877 -1.472093  1.721358
1959-06-30  2.34   2778.801   5.1  0.958215  1.617037 -0.685642
1959-09-30  2.74   2775.488   5.3  1.466274 -1.503125 -1.247412
1959-12-31  0.27   2785.204   5.6 -0.350741  0.298730 -1.938654
1960-03-31  2.31   2847.699   5.2 -0.544541 -1.201026 -0.245193
1960-06-30  0.14   2834.390   5.2 -0.207895 -2.083533 -1.136374
1960-09-30  2.70   2839.022   5.6 -0.648102  0.385712  0.712359
1960-12-31  1.21   2802.616   6.3  0.750663 -1.049285  0.591678
1961-03-31 -0.40   2819.264   6.8 -2.729431  0.317933  0.486536
1961-06-30  1.47   2872.005   7.0 -0.542326 -0.271943 -0.193465
1961-09-30  0.80   2918.419   6.8 -1.570660  2.011624  0.115840
1961-12-31  0.80   2977.830   6.2 -1.095856  1.749099  0.180717
1962-03-31  2.26   3031.241   5.6 -1.204816 -0.896555 -0.509060
1962-06-30  0.13   3064.709   5.5  0.500730 -0.474709  0.436206
1962-09-30  2.11   3093.047   5.6  1.379492  0.290913 -0.987553
1962-12-31  0.79   3100.563   5.5  0.014322  0.548519  0.520608
1963-03-31  0.53   3141.087   5.8  1.375142 -0.153599  0.114385
1963-06-30  2.75   3180.447   5.7  0.575837 -1.236940  1.579612
1963-09-30  0.78   3240.332   5.5 -2.719489  0.114516  1.027368
1963-12-31  2.46   3264.967   5.6 -0.728629  0.770726  0.358885
1964-03-31  0.13   3338.246   5.5 -0.777633 -0.525409  0.288665
1964-06-30  0.90   3376.587   5.2 -1.700424 -2.689190 -0.956353
1964-09-30  1.29   3422.469   5.0  0.029319 -0.574724 -2.500792
1964-12-31  2.05   3431.957   5.0  1.037442  1.123268  0.433927
1965-03-31  1.28   3516.251   4.9  1.338541  0.289026 -0.887498
1965-06-30  2.54   3563.960   4.7  1.366885  2.245591  0.235678
1965-09-30  0.89   3636.285   4.4  0.149515  0.344875  0.698727
1965-12-31  2.90   3724.014   4.1  0.093153  0.744506  0.572767
1966-03-31  4.99   3815.423   3.9  0.058013  0.423520 -0.737886
1966-06-30  2.10   3828.124   3.8  3.011197 -0.242716  1.472575
...          ...        ...   ...       ...       ...       ...
2002-06-30  1.56  11538.770   5.8  0.038347  1.388060 -0.428484
2002-09-30  2.66  11596.430   5.7  1.491195 -0.866267 -0.449508
2002-12-31  3.08  11598.824   5.8  2.012276 -1.951721  0.917595
2003-03-31  1.31  11645.819   5.9  1.443736 -1.064772  1.438180
2003-06-30  1.09  11738.706   6.2 -0.148972 -0.437413  0.389309
2003-09-30  2.60  11935.461   6.1 -0.426536  0.533119  0.312218
2003-12-31  3.02  12042.817   5.8  1.314191 -0.025952 -1.267970
2004-03-31  2.35  12127.623   5.7 -0.718064 -0.680210  0.318129
2004-06-30  3.61  12213.818   5.6 -1.566601  1.060304 -2.990728
2004-09-30  3.58  12303.533   5.4  0.541908  1.111213  0.471954
2004-12-31  2.09  12410.282   5.4 -0.116091  1.372303  0.820976
2005-03-31  4.15  12534.113   5.3 -0.903743 -0.289001  0.823369
2005-06-30  1.85  12587.535   5.1  0.607598  0.817519 -0.539960
2005-09-30  9.14  12683.153   5.0 -0.790762  0.806856 -1.984274
2005-12-31  0.40  12748.699   4.9 -0.655846  0.138485  2.102610
2006-03-31  2.60  12915.938   4.7  1.777580  0.462186  0.352988
2006-06-30  3.97  12962.462   4.7  1.401818  0.329945 -0.720076
2006-09-30 -1.58  12965.916   4.7  0.728812 -0.143196 -0.712273
2006-12-31  3.30  13060.679   4.4 -0.770847 -0.316741 -0.772838
2007-03-31  4.58  13099.901   4.5 -1.081326  1.380502  0.398789
2007-06-30  2.75  13203.977   4.5 -0.321086  0.416891  1.384001
2007-09-30  3.45  13321.109   4.7  1.101357 -0.564756 -1.629418
2007-12-31  6.38  13391.249   4.8 -0.527306  0.068665 -1.883100
2008-03-31  2.82  13366.865   4.9  1.409072  2.436707  0.158112
2008-06-30  8.53  13415.266   5.4  0.234440 -0.723400  1.245812
2008-09-30 -3.16  13324.600   6.0  0.007508  1.525763  0.374137
2008-12-31 -8.79  13141.920   6.9 -1.860111  1.110759 -1.302322
2009-03-31  0.94  12925.410   8.1 -0.039342  1.195749  0.726822
2009-06-30  3.37  12901.504   9.2  0.293209

In [72]:
pivoted2.reset_index().melt(['date'],['infl','realgdp'],col_level = 1 )

UnsortedIndexError: 'MultiIndex Slicing requires the index to be fully lexsorted tuple len (1), lexsort depth (0)'

In [74]:
pivoted2.reset_index()[['date','value']]

date value                 
item             infl    realgdp unemp
0    1959-03-31  0.00   2710.349   5.8
1    1959-06-30  2.34   2778.801   5.1
2    1959-09-30  2.74   2775.488   5.3
3    1959-12-31  0.27   2785.204   5.6
4    1960-03-31  2.31   2847.699   5.2
5    1960-06-30  0.14   2834.390   5.2
6    1960-09-30  2.70   2839.022   5.6
7    1960-12-31  1.21   2802.616   6.3
8    1961-03-31 -0.40   2819.264   6.8
9    1961-06-30  1.47   2872.005   7.0
10   1961-09-30  0.80   2918.419   6.8
11   1961-12-31  0.80   2977.830   6.2
12   1962-03-31  2.26   3031.241   5.6
13   1962-06-30  0.13   3064.709   5.5
14   1962-09-30  2.11   3093.047   5.6
15   1962-12-31  0.79   3100.563   5.5
16   1963-03-31  0.53   3141.087   5.8
17   1963-06-30  2.75   3180.447   5.7
18   1963-09-30  0.78   3240.332   5.5
19   1963-12-31  2.46   3264.967   5.6
20   1964-03-31  0.13   3338.246   5.5
21   1964-06-30  0.90   3376.587   5.2
22   1964-09-30  1.29   3422.469   5.0
23   1964-12-31  2.05   3431.957   5.0
24   1965-03-31  1.28   3516.251   4.9
25   1965-06-30  2.54   3563.960   4.7
26   1965-09-30  0.89   3636.285   4.4
27   1965-12-31  2.90   3724.014   4.1
28   1966-03-31  4.99   3815.423   3.9
29   1966-06-30  2.10   3828.124   3.8
..          ...   ...        ...   ...
173  2002-06-30  1.56  11538.770   5.8
174  2002-09-30  2.66  11596.430   5.7
175  2002-12-31  3.08  11598.824   5.8
176  2003-03-31  1.31  11645.819   5.9
177  2003-06-30  1.09  11738.706   6.2
178  2003-09-30  2.60  11935.461   6.1
179  2003-12-31  3.02  12042.817   5.8
180  2004-03-31  2.35  12127.623   5.7
181  2004-06-30  3.61  12213.818   5.6
182  2004-09-30  3.58  12303.533   5.4
183  2004-12-31  2.09  12410.282   5.4
184  2005-03-31  4.15  12534.113   5.3
185  2005-06-30  1.85  12587.535   5.1
186  2005-09-30  9.14  12683.153   5.0
187  2005-12-31  0.40  12748.699   4.9
188  2006-03-31  2.60  12915.938   4.7
189  2006-06-30  3.97  12962.462   4.7
190  2006-09-30 -1.58  12965.916   4.7
191  2006-12-31  3.30  13060.679   4.4
192  2007-03-31  4.58  13099.901   4.5
193  2007-06-30  2.75  13203.977   4.5
194  2007-09-30  3.45  13321.109   4.7
195  2007-12-31  6.38  13391.249   4.8
196  2008-03-31  2.82  13366.865   4.9
197  2008-06-30  8.53  13415.266   5.4
198  2008-09-30 -3.16  13324.600   6.0
199  2008-12-31 -8.79  13141.920   6.9
200  2009-03-31  0.94  12925.410   8.1
201  2009-06-30  3.37  12901.504   9.2
202  2009-09-30  3.56  12990.341   9.6

[203 rows x 4 columns]